# Convnet Model and Training

This aim of notebook is to define the siamese model that will be used for the Humpback Whale challenge. It will also run the model to trainin it agains our previously generated training and validation dataset and also create the results agains a test dataset.

## 0. Module Imports 

In [1]:
import numpy as np
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

## 1. Load Training, Validation and Test datasets

In [3]:
pickle_file = '../data/Siamese_dataset.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_X1_dataset = save['train_X1_dataset']
    train_X2_dataset = save['train_X2_dataset']
    train_labels = save['train_labels']
    valid_X1_dataset = save['valid_X1_dataset']
    valid_X2_dataset = save['valid_X2_dataset']
    valid_labels = save['valid_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_X1_dataset.shape, train_X2_dataset.shape, train_labels.shape)
    print('Validation set', valid_X1_dataset.shape, valid_X2_dataset.shape, valid_labels.shape)

Training set (31218, 44, 100) (31218, 44, 100) (31218,)
Validation set (5510, 44, 100) (5510, 44, 100) (5510,)


##  2. Declaration of Global Constants
Now we declare all the global constants in hee in orde to have them centralized.
This way we can easyly tune our model.

In [4]:
## Image parameters
NUM_CHANNELS = 1
IMAGE_HEIGHT = 44
IMAGE_WIDHT = 100

#Convnet parameters
BETA = 0.005
DEPTH_1 = 8

#Training parameters
BATCH_SIZE = 128

## 3. Reformat datasets into a TensorFlow-friendly shape:

- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.



In [8]:
# No need to reformat ?? TBD

## 4. Definition of auxiliar functions

### 4.1. Accuracy computation

In [9]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## 5. Network declaration
We will use tensorflow

In [4]:

siamese_model = tf.Graph()

with siamese_model.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
      tf.float32, shape=(batch_size, IMAGE_WIDHT, IMAGE_HEIGHT, NUM_CHANNELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    
    weights = {
      'h1': tf.Variable( tf.truncated_normal([image_size * image_size, hidden1_nodes]) ),
      'out': tf.Variable( tf.truncated_normal([hidden1_nodes, num_labels]) )
      }
    biases = {
        'h1' :  tf.Variable(tf.zeros([depth])),
        'out' :  tf.Variable(tf.zeros([depth]))
        }
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


SyntaxError: invalid syntax (<ipython-input-4-85cedc4c327b>, line 20)

## 6. Model training